***In this File I am going to Implement SVD and User Based, item Based Collaborative filtering collaborative filtering***

In [15]:
# In this I am going to implement SVD, KNN Algorithm to filter movies according to user ratings

In [1]:
# Downloading the Ratings Dataset
!wget https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/movielens_data/ratings.csv

--2022-05-29 09:17:10--  https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/movielens_data/ratings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2438266 (2.3M) [text/plain]
Saving to: ‘ratings.csv’

ratings.csv         100%[===================>]   2.33M  --.-KB/s    in 0.06s   

2022-05-29 09:17:11 (36.9 MB/s) - ‘ratings.csv’ saved [2438266/2438266]



In [2]:
# Downloading the Movies Dataset
!wget https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/movielens_data/movies.csv

--2022-05-29 09:17:11--  https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/movielens_data/movies.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 458390 (448K) [text/plain]
Saving to: ‘movies.csv’

movies.csv          100%[===================>] 447.65K  --.-KB/s    in 0.04s   

2022-05-29 09:17:11 (12.1 MB/s) - ‘movies.csv’ saved [458390/458390]



In [3]:
# Installing the Surprise Package
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 5.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633736 sha256=be2a3626677be40e22a0635560d236e9ee307581dd69e3b9ea957462f108b1fe
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [16]:
# importing all the necessary libraries
import numpy as np
import pandas as pd

# for converting Data into the Surprise Reader
from surprise import Dataset, Reader

# for finding accuracy of the model
from surprise.model_selection import  cross_validate

# for building SVD model
from surprise import SVD

# to create a deafault dictionary 
from collections import defaultdict

In [5]:
# Lets Read the Movies Dataset
movies = pd.read_csv('movies.csv')

# lets check the shape of the movies dataset
movies.shape

(9125, 3)

In [6]:
# lets check the head of the movies data
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# Lets Read the Ratings Dataset
ratings = pd.read_csv('ratings.csv')

# lets check the shape of the dataset
ratings.shape

(100004, 4)

In [8]:
# lets check the head of the ratings data
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [9]:
# Removing the Timestamp column from the Data, as the Surprise Library Accepts only three Columns
ratings = ratings.drop(['timestamp'], axis = 1)

# Now, i have to Create a CSV File for the new rating data, as the Surprise Library can only accept CSV Files as Input
# I have have to Specify the Header as None, as again the Surprise Library cannot take in Column Names
# I have have to set the Index also as False, Because the Surprise Library cannot Handle Indexes also.
ratings.to_csv("ratings_modified.csv", 
               header = None, 
               index = False)

In [10]:
# Surprise Reader can also read CSV files and not DataFrames

# lets take a Look at the Modified Dataset Before Feeding it to the Surprise Data Reader
x = pd.read_csv('ratings_modified.csv')
x.head()

,1,31,2.5
0,1,1029,3.0
1,1,1061,3.0
2,1,1129,2.0
3,1,1172,4.0
4,1,1263,2.0


In [18]:

# Lets First Specify the File Path and Reader Parameters Required for Loading the Data
file_path = 'ratings_modified.csv'
reader = Reader(line_format='user item rating', sep=',', rating_scale = (1,5))

# Lets Load the Dataset into the Surprise Reader, We cannot read this Dataset, as this is a Surprise Object. 
data = Dataset.load_from_file(file_path, reader=reader)



In [19]:
# Lets Build the Training Dataset
train = data.build_full_trainset()

In [20]:
# lets build the model

# n_factor = no. of factors considered while factorize the matrix
# n_epochs = no. of iterations required to train the model
algo = SVD(n_factors = 200, n_epochs = 30)

In [21]:
# lets check the Accuracy of the Model. For this we use Cross_validate function
# cross_validate - A resampling procedure used to evaluate machine learning models on alimited data sample


# Cross validation 
# this will take some time as it checking the accuracy of whole model
results = cross_validate(algo = algo, 
                         data = data, 
                         measures=['RMSE'], #root mean square error
                         cv = 5, # check accuracy of 5 different group of data 
                         return_train_measures=True)
    
print("Result RMSE : ",results['test_rmse'].mean())

Result RMSE :  0.9016964665651017


In [22]:
# accuracy is arround 90%
# Training the Model on our Dataset
algo.fit(train)

In [23]:
#  building the mapping
# first of all we will have to build a mapping of movie ID and movie titles because the surprise library will produce the results in the form of user IDs and movie IDs, which will be very difficult to understand and interpret

# So, lets Create a Dictionary to Map the Movie Id and Movie Names
movie_id_to_title_map = {} # empty dictionary

# mapping the movie ID with movie title
for m_id , title in zip(movies['movieId'].values , movies['title'].values):
    movie_id_to_title_map[str(m_id)] = title

# lets check the Dictionary that I Created
movie_id_to_title_map

{'1': 'Toy Story (1995)',
 '2': 'Jumanji (1995)',
 '3': 'Grumpier Old Men (1995)',
 '4': 'Waiting to Exhale (1995)',
 '5': 'Father of the Bride Part II (1995)',
 '6': 'Heat (1995)',
 '7': 'Sabrina (1995)',
 '8': 'Tom and Huck (1995)',
 '9': 'Sudden Death (1995)',
 '10': 'GoldenEye (1995)',
 '11': 'American President, The (1995)',
 '12': 'Dracula: Dead and Loving It (1995)',
 '13': 'Balto (1995)',
 '14': 'Nixon (1995)',
 '15': 'Cutthroat Island (1995)',
 '16': 'Casino (1995)',
 '17': 'Sense and Sensibility (1995)',
 '18': 'Four Rooms (1995)',
 '19': 'Ace Ventura: When Nature Calls (1995)',
 '20': 'Money Train (1995)',
 '21': 'Get Shorty (1995)',
 '22': 'Copycat (1995)',
 '23': 'Assassins (1995)',
 '24': 'Powder (1995)',
 '25': 'Leaving Las Vegas (1995)',
 '26': 'Othello (1995)',
 '27': 'Now and Then (1995)',
 '28': 'Persuasion (1995)',
 '29': 'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
 '30': 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 '31': 'Dangerou

In [24]:
# Now it Can ready to give predictions
# Realtime prediction

# example - let's check how much the user id - 2 would rate item id 25 ?
val = algo.predict(uid = '2', iid = '25')
print(movie_id_to_title_map[val[1]] , val[3])

Leaving Las Vegas (1995) 3.4865871798468184


In [25]:
# here is the rating user 2 give to the movie Leaving Las vegas

In [26]:
# now let create a function which can give us the recommendation for Top movies for all the users present on the database

# Get Top n Predictions

def get_top_n(predictions, n=10):# n - no. of movies you need to recommend
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [27]:
# building a Anti test set
# Anti test set - It have all the data which is not related to that particular user
# Build an Iterable Testset, Direct predictions on Train would throw errors 

# I build the anti test for all the users that have there personalized movie recommendations

testdata = train.build_anti_testset()
predictions = algo.test(testdata)
top_n = get_top_n(predictions, n = 10)

In [28]:
# let's Create a Function to fetch all the movies Watched By the users

def movies_watched(user_df, user_id, item_map):
    user_df = user_df[user_df.iloc[:,0] == user_id]
    for movie, rating in zip(user_df.iloc[:,1].values, user_df.iloc[:,2].values):
        print(item_map[str(movie)], rating)

In [29]:
# Lets Create a Function to Predict Movies to the Users based on the Movies Watched Previously
def UserPredictions(user_id , top_n , item_map):
    print("Predictions for User Id : " , user_id)
    user_ratings = top_n[user_id]
    for item_id , rating in user_ratings :
        print(item_map[item_id] , " : " , rating)

In [30]:
# Now let's check the movies watched by user 1
# giving the rating datframe which contains user's rating details, userId and mapping that I created earlier
movies_watched(ratings, 1, movie_id_to_title_map)

Dangerous Minds (1995) 2.5
Dumbo (1941) 3.0
Sleepers (1996) 3.0
Escape from New York (1981) 2.0
Cinema Paradiso (Nuovo cinema Paradiso) (1989) 4.0
Deer Hunter, The (1978) 2.0
Ben-Hur (1959) 2.0
Gandhi (1982) 2.0
Dracula (Bram Stoker's Dracula) (1992) 3.5
Cape Fear (1991) 2.0
Star Trek: The Motion Picture (1979) 2.5
Beavis and Butt-Head Do America (1996) 1.0
French Connection, The (1971) 4.0
Tron (1982) 4.0
Gods Must Be Crazy, The (1980) 3.0
Willow (1988) 2.0
Antz (1998) 2.0
Fly, The (1986) 2.5
Time Bandits (1981) 1.0
Blazing Saddles (1974) 3.0


In [31]:
# These are the movies that User 1 Watched 

In [32]:
# Now checking how SVD recommends user 1 using other users ratings

In [33]:
UserPredictions('1', top_n, movie_id_to_title_map)

Predictions for User Id :  1
Grand Day Out with Wallace and Gromit, A (1989)  :  3.7392906040881613
Shawshank Redemption, The (1994)  :  3.7153866085813276
Pulp Fiction (1994)  :  3.7088536438997237
Modern Times (1936)  :  3.705981324618177
Lawrence of Arabia (1962)  :  3.671488998136623
Stand by Me (1986)  :  3.5942903316898245
African Queen, The (1951)  :  3.5630876428037768
Christmas Story, A (1983)  :  3.5548928506830713
Band of Brothers (2001)  :  3.525537576766787
Boot, Das (Boat, The) (1981)  :  3.5077530310407785


In [34]:
# These are the recommendation for user 1

**KNN**

In [35]:
# User Based Collaborative filtering

In [47]:
#using the same train dataset

# lets get the Number of Users and Items
print('Number of users in the Database :', train.n_users)
print('Number of items in the Database :', train.n_items)

Number of users in the Database : 671
Number of items in the Database : 9066


In [37]:
# I am going to use KNN algorithm

In [40]:
# KNN algorithm - It is based on assumption that If two data points are close to each other then those data points are similar to each other.

# Lets Build the Model

from surprise import KNNWithMeans

# User Based collaborative Filtering that is why user_based = true.
my_sim_option = {'name':'pearson', 'user_based':True}

# KNN model as backend 
# k is no. of neighbours which are need to be recommend
algo = KNNWithMeans(k = 15, min_k = 5, 
                    sim_options = my_sim_option, 
                    verbose = True)

In [39]:
# lets check the Accuracy of the Model. For this we use Cross_validate function
# cross_validate - A resampling procedure used to evaluate machine learning models on alimited data sample
from surprise.model_selection import  cross_validate

# Cross validation 
# this will take some time as it checking accuracy of whole model
results = cross_validate(algo = algo, 
                         data = data, 
                         measures=['RMSE'], #root mean square error
                         cv = 5, # check accuracy of 5 different group of data 
                         return_train_measures=True)
    
print(results['test_rmse'].mean())

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
0.9190043276401324


In [46]:
# accuracy is arround 92%
# Training the Model on our Dataset
algo.fit(train)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [43]:
# first of all we will have to build a mapping of movie ID and movie titles 
# because the surprise library will produce the results in the form of user IDs and movie IDs, 
# which will be very difficult to understand and interpret

# So, lets Create a Dictionary to Map the Movie Id and Movie Names
movie_id_to_title_map = {} # empty dictionary

# mapping the movie ID with movie title
for m_id , title in zip(movies['movieId'].values , movies['title'].values):
    movie_id_to_title_map[str(m_id)] = title

# lets check the Dictionary that we Created
movie_id_to_title_map

{'1': 'Toy Story (1995)',
 '2': 'Jumanji (1995)',
 '3': 'Grumpier Old Men (1995)',
 '4': 'Waiting to Exhale (1995)',
 '5': 'Father of the Bride Part II (1995)',
 '6': 'Heat (1995)',
 '7': 'Sabrina (1995)',
 '8': 'Tom and Huck (1995)',
 '9': 'Sudden Death (1995)',
 '10': 'GoldenEye (1995)',
 '11': 'American President, The (1995)',
 '12': 'Dracula: Dead and Loving It (1995)',
 '13': 'Balto (1995)',
 '14': 'Nixon (1995)',
 '15': 'Cutthroat Island (1995)',
 '16': 'Casino (1995)',
 '17': 'Sense and Sensibility (1995)',
 '18': 'Four Rooms (1995)',
 '19': 'Ace Ventura: When Nature Calls (1995)',
 '20': 'Money Train (1995)',
 '21': 'Get Shorty (1995)',
 '22': 'Copycat (1995)',
 '23': 'Assassins (1995)',
 '24': 'Powder (1995)',
 '25': 'Leaving Las Vegas (1995)',
 '26': 'Othello (1995)',
 '27': 'Now and Then (1995)',
 '28': 'Persuasion (1995)',
 '29': 'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
 '30': 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 '31': 'Dangerou

In [44]:
# Let's try to know what rating the user 1 will give to the movie 25
# Realtime prediction

# example - let's check how much the user id - 2 would rate item id 25 ?
val = algo.predict(uid = '2', iid = '25')
print(movie_id_to_title_map[val[1]] , val[3])

Leaving Las Vegas (1995) 3.798568336128513


In [48]:
# building a Anti test set as i also do earlier
# Anti test set - It have all the data which is not related to that particular user
# Build an Iterable Testset, Direct predictions on Train would throw errors 

testdata = train.build_anti_testset()
predictions = algo.test(testdata)
top_n = get_top_n(predictions, n = 10)

In [49]:
# let's check movies watched by user 1
movies_watched(ratings , 1 , movie_id_to_title_map)

Dangerous Minds (1995) 2.5
Dumbo (1941) 3.0
Sleepers (1996) 3.0
Escape from New York (1981) 2.0
Cinema Paradiso (Nuovo cinema Paradiso) (1989) 4.0
Deer Hunter, The (1978) 2.0
Ben-Hur (1959) 2.0
Gandhi (1982) 2.0
Dracula (Bram Stoker's Dracula) (1992) 3.5
Cape Fear (1991) 2.0
Star Trek: The Motion Picture (1979) 2.5
Beavis and Butt-Head Do America (1996) 1.0
French Connection, The (1971) 4.0
Tron (1982) 4.0
Gods Must Be Crazy, The (1980) 3.0
Willow (1988) 2.0
Antz (1998) 2.0
Fly, The (1986) 2.5
Time Bandits (1981) 1.0
Blazing Saddles (1974) 3.0


In [50]:
# now let's check our final result: movies recommended by user 1 
UserPredictions('1' , top_n , movie_id_to_title_map)

Predictions for User Id :  1
All the President's Men (1976)  :  3.983289650565472
Kids in the Hall: Brain Candy (1996)  :  3.8002679753118342
Diva (1981)  :  3.7906101850318987
There Will Be Blood (2007)  :  3.7898802303451076
Scott Pilgrim vs. the World (2010)  :  3.781808833120466
Paradise Lost: The Child Murders at Robin Hood Hills (1996)  :  3.7698992239149804
When We Were Kings (1996)  :  3.7179327363818
Payback (1999)  :  3.7057743197750863
True Grit (2010)  :  3.6701056262957192
My Life in Pink (Ma vie en rose) (1997)  :  3.6355808931561038


In [ ]:
# Item Based Filtering

In [51]:
# Lets Build the Model

from surprise import KNNWithMeans

# Item Based collaborative Filtering.
my_sim_option = {'name':'pearson', 'user_based':False}

# KNN model as backend 
# k is no. of neighbours which are need to be recommend
algo = KNNWithMeans(k = 15, min_k = 5, 
                    sim_options = my_sim_option, 
                    verbose = True)

In [52]:
# lets check the Accuracy of the Model. For this we use Cross_validate function
# cross_validate - A resampling procedure used to evaluate machine learning models on alimited data sample
from surprise.model_selection import  cross_validate

# Cross validation 
# this will take Time as it is checking accuracy of the whole model
results = cross_validate(algo = algo, 
                         data = data, 
                         measures=['RMSE'], #root mean square error
                         cv = 5, # check accuracy of 5 different group of data 
                         return_train_measures=True)
    
print(results['test_rmse'].mean())

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
0.9433330620145742


In [53]:
# accuracy is arround 94%
# more accuracy then user based collaborative filtering
# Lets Training the Model on our Dataset
algo.fit(train)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [54]:
# first of all we will have to build a mapping of movie ID and movie titles because the surprise library will produce the results in the form of user IDs and movie IDs, which will be very difficult to understand and interpret

# So, lets Create a Dictionary to Map the Movie Id and Movie Names
movie_id_to_title_map = {} # empty dictionary

# mapping the movie ID with movie title
for m_id , title in zip(movies['movieId'].values , movies['title'].values):
    movie_id_to_title_map[str(m_id)] = title

# lets check the Dictionary that we Created
movie_id_to_title_map

{'1': 'Toy Story (1995)',
 '2': 'Jumanji (1995)',
 '3': 'Grumpier Old Men (1995)',
 '4': 'Waiting to Exhale (1995)',
 '5': 'Father of the Bride Part II (1995)',
 '6': 'Heat (1995)',
 '7': 'Sabrina (1995)',
 '8': 'Tom and Huck (1995)',
 '9': 'Sudden Death (1995)',
 '10': 'GoldenEye (1995)',
 '11': 'American President, The (1995)',
 '12': 'Dracula: Dead and Loving It (1995)',
 '13': 'Balto (1995)',
 '14': 'Nixon (1995)',
 '15': 'Cutthroat Island (1995)',
 '16': 'Casino (1995)',
 '17': 'Sense and Sensibility (1995)',
 '18': 'Four Rooms (1995)',
 '19': 'Ace Ventura: When Nature Calls (1995)',
 '20': 'Money Train (1995)',
 '21': 'Get Shorty (1995)',
 '22': 'Copycat (1995)',
 '23': 'Assassins (1995)',
 '24': 'Powder (1995)',
 '25': 'Leaving Las Vegas (1995)',
 '26': 'Othello (1995)',
 '27': 'Now and Then (1995)',
 '28': 'Persuasion (1995)',
 '29': 'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
 '30': 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 '31': 'Dangerou

In [55]:
# Let's try to know what rating the user 1 will give to the movie 25
# Realtime prediction

# example - let's check how much the user id - 1 would rate item id 31 ?
val = algo.predict(uid = '1', iid = '31')
print(movie_id_to_title_map[val[1]] , val[3])

Dangerous Minds (1995) 2.1943918880284228
